## 01/03/2019

Extracting features from each segments.

In [1]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

# Import necessary modules. Set settings. Import data.
import math
import numpy as np
import pandas as pd
import random
import pywt
import matplotlib.pyplot as plt
from statsmodels.robust import mad
from tsfresh.feature_extraction import feature_calculators
from FeatureExtraction.feature_tools import detect_peaks
from IPython.display import display, clear_output, HTML

import pdb

plt.style.use('default')

X = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/Data/X_all_channel_labels.pkl')

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Remove bad files with bad labels
X = X[~(X['Label 1']=='-1') & ~(X['Label 2']=='-1')]
X.head()

,Channel,Coupling Interval,Data,Patient,S1/S2,Type,Label 1,Label 2
0,CS1-2,340,"[-636, -617, -652, -560, -482, -415, -383, -46...",1,S2,af,0,0
1,CS1-2,340,"[-903.0, -873.0, -935.0, -941.0, -910.0, -845....",1,S1,af,NaN,NaN
2,CS1-2,340,"[-931.0, -896.0, -896.0, -906.0, -858.0, -839....",1,S1,af,NaN,NaN
3,CS3-4,340,"[472, 464, 491, 523, 553, 706, 1019, 1404, 164...",1,S2,af,0,0
4,CS3-4,340,"[298.0, 292.0, 303.0, 311.0, 299.0, 395.0, 451...",1,S1,af,NaN,NaN


In [3]:
# A shitty conduction delay detector
def get_delay(x, amp_thresh=None, set_thresh=False):
    if (set_thresh==True):
        if any(abs(x)>amp_thresh):
            return np.argmax(abs(x)>amp_thresh)
        else:
            return len(x)
    else:    
        return np.argmax(abs(x)>(max(abs(x))/2))
    
def denoise(x):
    # Obtain Daubechies N=6 wavelet coefficients
    waveletCoefs = pywt.wavedec(x, 'db7', mode='per')

    # Throw away coefficients corresponding to noise
    sigma = mad(waveletCoefs[-1])
    uThresh = 1*sigma*np.sqrt(2*np.log(len(x)))
    denoised = waveletCoefs[:]
    denoised[1:] = (pywt._thresholding.hard(i, value=uThresh) for i in denoised[1:])

    # Reconstruct the original signal
    xDenoised = pywt.waverec(denoised, 'db7', mode='per')

    return xDenoised

def get_peaks(x, height_thresh, scale_amp=None, set_scale=False, plot = False):
    x = np.array(x)
    
    # Get height_thresh
    if set_scale:
        height_thresh = height_thresh*scale_amp
    else:
        height_thresh = height_thresh*max(abs(x))
    
    # Denoise x
    xdn = denoise(x)

    # Detect peaks using detect_peaks
    pos_peak_idx = detect_peaks(xdn, mph=height_thresh, threshold = 0)
    neg_peak_idx = detect_peaks((-xdn), mph=height_thresh, threshold = 0)
    peak_idx = np.concatenate([pos_peak_idx, neg_peak_idx])
    peak_idx = np.sort(peak_idx)
    # Edge indeces aren't detected
    peak_idx = peak_idx[(peak_idx != 0) & (peak_idx != (len(xdn)-1))]

    new_peak_idx = []
    peak_amp = []
    if (len(peak_idx) > 0):
        new_peak_idx.append(peak_idx[0])
        mp_thresh = 0.2*max(abs(x))
        for i in range(len(peak_idx)-1):
            idx = peak_idx[i]
            idx_next = peak_idx[i+1]
            mid_point = int((idx_next+idx)/2)
            if (max([abs(x[idx_next]-x[mid_point]), abs(x[idx]-x[mid_point])]) > mp_thresh):
                new_peak_idx.append(idx_next)

        peak_idx = np.array(new_peak_idx)
        peak_amp = x[peak_idx]

    if plot == True:
        fig, [ax1] = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(8,8))
        ax1.plot(x, 'b' , xdn, 'r--', peak_idx, peak_amp, 'kx')
        #plt.title(fileName)
        ax1.set_xlabel('Sample')
        ax1.set_ylabel('Normalised amplitude')
        ax1.legend(['Original segment', 'Denoised segment', 'Detected peaks'])

        plt.draw()
        plt.waitforbuttonpress(0) # this will wait for indefinite time
        plt.close(fig)


    return peak_idx, peak_amp

def sample_entropy(U, m, r):

    def _maxdist(x_i, x_j):
        result = max([abs(ua-va) for ua, va in zip(x_i, x_j)])
        return result

    def _phi(m):
        x = np.zeros([N,m-1])
        for i in range(N-m+1):
            x[i,:] = U[i:i+m-1]

        C = 0
        for i in range(len(x)):
            for j in range(len(x)):
                if i != j:
                    if _maxdist(x[i,:], x[j,:]) <= r:
                        C = C + 1

        return C

    U = U/max(abs(U))
    N = len(U)

    return -np.log(_phi(m+1)/_phi(m))

def percentage_fractionation(x, peak_idxs, thresh=0.01, sr=1000):
    # Get peak indexes and amplitude
    peak_idx_diffs = np.diff(peak_idxs)
    frac_time = 0
    frac_time = np.sum(peak_idx_diffs[peak_idx_diffs < thresh*sr])
    prcnt_frac = (frac_time/len(x))*100
    return prcnt_frac

def get_local_sample_entropy(x, centre_idx, width, m=2, r=0.05):
    # Ensure width is odd
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return sample_entropy(x[:width+1], m, r)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return sample_entropy(x[len(x)-1-width:], m, r)
    else:
        return sample_entropy(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)], m, r)
    
def get_location_of_max_energy(x, M=14):
    v = np.ones(M)
    x_ = np.convolve(abs(x), v)
    return (np.argmax(x_) + math.floor(M/2))
        
def get_local_peaks(x, centre_idx, width=25, height_thresh=0.1):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_peaks(x[:width+1], height_thresh)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_peaks(x[len(x)-1-width:], height_thresh)
    else:
        return get_peaks(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)], height_thresh)
    
def get_pse(x):
    x_fft = np.fft.rfft(x)
    x_P = (1/len(x_fft))*np.absolute(x_fft)**2
    x_p = x_P/sum(x_P)
    pse = np.sum([(-p*np.log2(p)) for p in x_p])
    return pse

def get_local_pse(x, centre_idx, width=50):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_pse(x[:width+1])
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_pse(x[len(x)-1-width:])
    else:
        return get_pse(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)])
    
def get_spectral_centroid(x):
    x_fft = np.fft.rfft(x)
    x_spectrum = np.absolute(x_fft)
    normalized_spectrum = x_spectrum/sum(x_spectrum)
    normalized_frequencies = np.arange(0, len(x_spectrum), 1)
    return sum(normalized_frequencies * normalized_spectrum)

def get_local_spectral_centroid(x, centre_idx, width=50):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_spectral_centroid(x[:width+1])
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_spectral_centroid(x[len(x)-1-width:])
    else:
        return get_spectral_centroid(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)])

In [5]:
def get_feature_dict(x, thresh_cd=None, set_thresh_cd=False, thresh_peaks=None, set_thresh_peaks=False, show_peaks=False, col_prefix = ''):
    feature_dict = {}
    height_thresh=0.1
    
    # Try normalising x first. Features extracted entirely on shape. Include the max value in features
    feature_dict[col_prefix + 'Maximum Absolute Value'] = max(abs(x))
    x = x/max(abs(x))
    thresh_cd = thresh_cd/max(abs(x))
    thresh_peaks = thresh_peaks/max(abs(x))

    # First features are hand-engineering
    if set_thresh_cd:
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=True'] = get_delay(x, thresh_cd, set_thresh_cd)
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=False'] = get_delay(x)
    else:
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=False'] = get_delay(x)

    if set_thresh_peaks:
        peaks = get_peaks(x, height_thresh, thresh_peaks, set_thresh_peaks, plot=False)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=True'] = len(peaks[0])
            
        peaks = get_peaks(x, height_thresh)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=False'] = len(peaks[0])
    else:
        peaks = get_peaks(x, height_thresh)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=False'] = len(peaks[0])
    
    peaks = get_peaks(x, height_thresh)
    feature_dict[col_prefix + 'Percentage Fractionation: thresh=0.01'] = percentage_fractionation(x, peaks[0], thresh=0.01)
    
    try:
        middle_peak_idx = peaks[0][math.ceil(len(peaks[0])/2)-1]
    except:
        middle_peak_idx=0

    # Denoise x and see if performance is affected
    x = denoise(x)
    
#     # Hand engineered features
#     feature_dict[col_prefix + 'Sample Entropy Around Middle Peak: width=40 r=0.05'] = get_local_sample_entropy(x, middle_peak_idx, 40, m=2, r=0.05)
#     feature_dict[col_prefix + 'Sample Entropy Around Middle Peak: width=40 r=0.025'] = get_local_sample_entropy(x, middle_peak_idx, 40, m=2, r=0.025)    
#     feature_dict[col_prefix + 'Sample Entropy Around Middle Peak: width=20 r=0.05'] = get_local_sample_entropy(x, middle_peak_idx, 20, m=2, r=0.05)
#     feature_dict[col_prefix + 'Sample Entropy Around Middle Peak: width=20 r=0.025'] = get_local_sample_entropy(x, middle_peak_idx, 20, m=2, r=0.025)    
#     feature_dict[col_prefix + 'Sample Entropy Around Middle Peak: width=60 r=0.05'] = get_local_sample_entropy(x, middle_peak_idx, 60, m=2, r=0.05)
#     feature_dict[col_prefix + 'Sample Entropy Around Middle Peak: width=60 r=0.025'] = get_local_sample_entropy(x, middle_peak_idx, 60, m=2, r=0.025)
#     feature_dict[col_prefix + 'Sample Entropy'] = sample_entropy(x, m=2, r=0.025)
    max_energy_idx = get_location_of_max_energy(x)
    feature_dict[col_prefix + 'Location of Maximum Energy: M=14'] = max_energy_idx
    feature_dict[col_prefix + 'Sample Entropy Around Max Energy: width=60 r=0.025'] = get_local_sample_entropy(x, max_energy_idx, 60, m=2, r=0.025)
    min_idx = np.argmin(x)
    max_idx = np.argmax(x)
    feature_dict[col_prefix + 'Peaks Between Min and Max'] = len([i for i in peaks[0] if ((i > min_idx) & (i < max_idx))])
    feature_dict[col_prefix + 'Power Spectral Entropy'] = get_pse(x)
    feature_dict[col_prefix + 'Spectral Centroid'] = get_spectral_centroid(x)
    feature_dict[col_prefix + 'Power Spectral Entropy Around Maximum Energy'] = get_local_pse(x, max_energy_idx, width=30)
    feature_dict[col_prefix + 'Spectral Centroid Around Maximum Energy'] = get_local_spectral_centroid(x, max_energy_idx, width=30)
    



    # Abritrary selection of features from TSFresh
    erbc = feature_calculators.energy_ratio_by_chunks(x, [{'num_segments':10, 'segment_focus':3}, {'num_segments':10, 'segment_focus':2}])
    feature_dict[col_prefix + 'Energy Ratio by Chunks: num_segments=10 segment_focus=2'] = erbc[1][1]
#     feature_dict[col_prefix + 'Approximate Entropy: m=2 r=0.1'] = feature_calculators.approximate_entropy(x, 2, 0.2)
#     feature_dict[col_prefix + 'Sample Entropy'] = feature_calculators.sample_entropy(x)
    feature_dict[col_prefix + 'Approximate Entropy: m=3 r=0.7'] = feature_calculators.approximate_entropy(x, 3, 0.7)
    feature_dict[col_prefix + 'Ratio Beyond 5xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 5)
    # A fraction q of the mass lies to the left of i. (Alternative to conduction delay?)
    imq = feature_calculators.index_mass_quantile(x, [{'q': 0.6}, {'q': 0.4}])
    feature_dict[col_prefix + 'Index Mass Quantile: q=0.6'] = imq[0][1]
    feature_dict[col_prefix + 'Standard Deviation'] = np.std(x)
    feature_dict[col_prefix + 'Normalised Standard Deviation'] = np.std(x/max(abs(x)))

    return feature_dict

In [6]:
cs12_features_list = []
cs34_features_list = []
cs56_features_list = []

for i,row in X[X['S1/S2']=='S2'].iterrows():
    clear_output(wait=True)
    display('Extracting Features: ' + str(round((i/X[X['S1/S2']=='S2'].index[-1])*100, 3)) + '%')
    
    # Get the patients response to the first S1 stimuli as the reference response
    ref_response = X[(X['S1/S2']=='S1') & (X['Patient']==row['Patient']) & (X['Type']==row['Type'])
                    ].sort_values(by=['Coupling Interval'], ascending=False).iloc[1]
    
    ref_feature_dict = get_feature_dict(ref_response['Data'], 
                                        thresh_cd=max(abs(ref_response['Data']))/5, 
                                        set_thresh_cd=False, 
                                        thresh_peaks=max(abs(ref_response['Data'])), 
                                        set_thresh_peaks=True, 
                                        show_peaks=False, 
                                        col_prefix = '')
    
    feature_dict = get_feature_dict(row['Data'],
                                    thresh_cd=max(abs(ref_response['Data']))/5, 
                                    set_thresh_cd=False, 
                                    thresh_peaks=max(abs(ref_response['Data'])), 
                                    set_thresh_peaks=True, 
                                    show_peaks=False, 
                                    col_prefix = '')
    
    for k, v in feature_dict.items():
        feature_dict[k] = v - ref_feature_dict[k]
        
    feature_dict['Label 1'] = row['Label 1']
    feature_dict['Label 2'] = row['Label 2']
    
    channel = row['Channel']
    if channel=='CS1-2':
        cs12_features_list.append(feature_dict)
    elif channel=='CS3-4':
        cs34_features_list.append(feature_dict)
    else:
        cs56_features_list.append(feature_dict)
        
cs12_features = pd.DataFrame(cs12_features_list)
cs34_features = pd.DataFrame(cs34_features_list)
cs56_features = pd.DataFrame(cs56_features_list)

'Extracting Features: 100.0%'

In [11]:
cs12_features.to_csv('/Users/matthewashman/github/MasterProject2018/FeatureSelection\cs12_features_for_mrmr.csv')
cs34_features.to_csv('/Users/matthewashman/github/MasterProject2018/FeatureSelection\cs34_features_for_mrmr.csv')
cs56_features.to_csv('/Users/matthewashman/github/MasterProject2018/FeatureSelection\cs56_features_for_mrmr.csv')